In [ ]:
ls /kaggle/input/shopee-utils-updated/

In [ ]:
from datetime import datetime,timedelta
start_time = datetime.now()

In [ ]:
import sys
import os
import pandas as pd
import gc
sys.path.insert(0,'../input/shopee-utils-updated-0-9/')
test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
test_df['image_path'] = '../input/shopee-product-matching/test_images/'+ test_df.image
SIMULATE = False
if test_df.shape[0]<5 and SIMULATE:
    train_df1 = pd.read_csv('../input/shopee-product-matching/train.csv')
    train_df2 = pd.read_csv('../input/shopee-product-matching/train.csv')
    train_df1['posting_id'] += 'a'
    train_df2['posting_id'] += 'b'
    test_df = pd.concat([train_df1,train_df2]).reset_index(drop=True)
    test_df['image_path'] = '../input/shopee-product-matching/train_images/'+ test_df.image
    del train_df1,train_df2
    gc.collect()
print(test_df.shape)
test_df.head(5)

In [ ]:
if SIMULATE:
    images_path = '../input/shopee-product-matching/train_images/'
else:
    images_path = '../input/shopee-product-matching/test_images/'

In [ ]:
!nvidia-smi

In [ ]:
from numba import cuda 
device = cuda.get_current_device()

## Image Models - EFFICIENTNET

In [ ]:
!pip install -U /kaggle/input/kerasapplications -q
!pip install -U /kaggle/input/efficientnet/efficientnet-master -q

In [ ]:
from shopee_image_utils import *

In [ ]:
IMAGE_SIZE = [640,640]
N_CLASSES = 11014
KNN = min(50,test_df.shape[0])
model_weights = {
    '../input/shopee-arcface-training-new-640/EF0_fold0_epoch15.h5':1,
    '../input/shopee-arcface-full-data-b2/EF2_fold0_epoch14.h5':1,
    '../input/shopee-arcface-full-data/EF0_fold0_epoch13.h5':1,
    '../input/shopee-arcface-training-new-b1/EF1_fold0_epoch15.h5':1,
}
IMAGE_SIZES = {
    '../input/shopee-arcface-training-new-640/EF0_fold0_epoch15.h5':[640,640],
    '../input/shopee-arcface-full-data-b2/EF2_fold0_epoch14.h5':[512,512],
    '../input/shopee-arcface-full-data/EF0_fold0_epoch13.h5':[512,512],
    '../input/shopee-arcface-training-new-b1/EF1_fold0_epoch15.h5':[512,512],
}
for file in model_weights:
    assert os.path.exists(file)
eff_neighbours = get_image_neighbours(model_weights, test_df, IMAGE_SIZES=IMAGE_SIZES, KNN=KNN)
eff_neighbours.columns = ['posting_id1','posting_id2','eff_distance']

In [ ]:
!nvidia-smi

In [ ]:
device.reset()

In [ ]:
!nvidia-smi

In [ ]:
eff_neighbours.to_csv('/tmp/eff_neighbours.csv',index=False)

In [ ]:
del eff_neighbours
gc.collect()

In [ ]:
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

In [ ]:
print(datetime.now()-start_time)

## Image Models - NFNET

In [ ]:
# from shopee_nfnet_utils import *

In [ ]:
# class Config1:
    
#     DATA_DIR = images_path
#     TEST_CSV = test_df
    
#     MODEL_PATH = '../input/shopee-nfnet-models/eca_nfnet_mish_8217.pt'

#     IMG_SIZE = 512
#     MEAN = [0.485, 0.456, 0.406]
#     STD = [0.229, 0.224, 0.225]
#     BATCH_SIZE = 48

#     NUM_WORKERS = 4
#     DEVICE = 'cuda'

#     CLASSES = 11014
#     SCALE = 30 
#     MARGIN = 0.5

#     MODEL_NAME = 'eca_nfnet_l0'
#     FC_DIM = 512

#     pool = 'gem' #['adaptive2d', 'gem', 'identity']
#     optimizer = 'adamw' # ['adamw', 'sgd', 'fused_sgd', 'ranger']
#     p_trainable = True
#     weight_decay = 0
#     mish = True
#     use_fc = True
# Config2 = None

In [ ]:
# class Config2:
    
#     DATA_DIR = images_path
#     TEST_CSV = test_df
    
#     MODEL_PATH = '../input/shopee-nfnet-models/eca_nfnet_mish_false_8205.pt'

#     IMG_SIZE = 512
#     MEAN = [0.485, 0.456, 0.406]
#     STD = [0.229, 0.224, 0.225]
#     BATCH_SIZE = 48

#     NUM_WORKERS = 4
#     DEVICE = 'cuda'

#     CLASSES = 11014
#     SCALE = 30 
#     MARGIN = 0.5

#     MODEL_NAME = 'eca_nfnet_l0'
#     FC_DIM = 512

#     pool = 'gem' #['adaptive2d', 'gem', 'identity']
#     optimizer = 'adamw' # ['adamw', 'sgd', 'fused_sgd', 'ranger']
#     p_trainable = True
#     weight_decay = 0
#     mish = False
#     use_fc = True


In [ ]:
# nfnet_neighbours = get_nfnet_neighbours(Config1,Config2,KNN=KNN)

In [ ]:
# nfnet_neighbours.to_csv('/tmp/nfnet_neighbours.csv',index=False)

In [ ]:
# del nfnet_neighbours,Config1,Config2
# gc.collect()

In [ ]:
!nvidia-smi

In [ ]:
print(datetime.now()-start_time)

## Clip Model

In [ ]:
# import sys
# !cp -r ../input/openai-clip/CLIP/CLIP-main /tmp/
# # Kaggle likes to unpack .gz files in datasets... so we have to pack it back
# !gzip -c /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt > /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt.gz
# sys.path.append('/tmp/CLIP-main')
# !pip install ../input/openai-clip/ftfy-5.9/ftfy-5.9 \
#              ../input/openai-clip/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl \
#              ../input/openai-clip/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl \
#              ../input/faiss-163/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
# from clip_utils import *

In [ ]:
# clip_model = ClipModel('../input/shopee-clip-vfnet-fold0/model_8.bin',df=test_df,images_path=images_path)
# clip_neighbours = clip_model.get_neighbours()

In [ ]:
# clip_neighbours.to_csv('/tmp/clip_neighbours.csv')

In [ ]:
# del clip_model
# gc.collect()

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# !nvidia-smi

In [ ]:
# print(datetime.now()-start_time)

## Text Models

In [ ]:
!pip install -U /kaggle/input/sentence-transformers/sentence-transformers-master/ -q

In [ ]:
from shopee_text_arcface_utils import *
from shopee_text_utils import *

In [ ]:
data = pd.read_csv('../input/shopee-product-matching/train.csv')
all_label_groups = data.label_group.value_counts().reset_index()
arcface_folds = 11014-(all_label_groups.index%5).value_counts()
arcface_folds.loc[-1] = 11014
def get_arcface_model(wt,fold):
    ############# ArcFace Model
    model_params = {'n_classes': arcface_folds.loc[fold], #-1 for full data model
     'model_name': '/kaggle/input/sentence-transformer-models/paraphrase-xlm-r-multilingual-v1/0_Transformer',
     'pooling': 'clf',
     'use_fc': False,
     'fc_dim': 512,
     'dropout': 0.0,
     'loss_module': 'arcface',
     's': 30.0,
     'margin': 0.5,
     'ls_eps': 0.0,
     'theta_zero': 0.785}
    arface_model_instance = arface_model(model_params,model_weights=wt)
    return arface_model_instance

In [ ]:
MODELS = [
    '../input/shopee-sbert-mbml-loss-fold-0/Bert-Shopee-epoch56/epoch56/',
    '../input/shopee-sbert-mbml-loss-fold-1/Bert-Shopee-epoch56/epoch56/',
    '../input/shopee-sbert-mbml-loss-fold-2/Bert-Shopee-epoch55/epoch55/',
    '../input/shopee-sbert-mbml-loss-fold-3/Bert-Shopee-epoch46/epoch46/',
    '../input/shopee-sbert-mbml-loss-fold-4/Bert-Shopee-epoch57/epoch57/', 
    '../input/shopee-models-mnr-loss-full-data-epoch-69/epoch69/',
    '../input/shopee-models-mnr-loss-fold-1/epoch60/',
    '../input/shopee-models-mnr-loss-fold-2/epoch69/',
    '../input/shopee-models-mnr-loss-fold-4/epoch62/'
    
]
WEIGHTS = []

for model in MODELS:
    if 'mbml' in model:
        WEIGHTS.append(0.6)
    elif 'mnr' in model:
        WEIGHTS.append(0.4)

MODELS += [ '../input/shopee-sbert-mbml-loss-full-data/Bert-Shopee-epoch50/epoch50/','../input/shopee-models-mnr-loss-all-folds-epoch-69/epoch69/']
WEIGHTS += [1.2,0.8]

arcface = [
    '../input/shopee-text-arcface-loss-fold-0/sentence_transfomer_xlm_best_loss_num_epochs_70_arcface.bin',
    '../input/shopee-text-arcface-loss-fold-1/sentence_transfomer_xlm_best_loss_num_epochs_70_arcface.bin',
    '../input/shopee-text-arcface-loss-fold-3/sentence_transfomer_xlm_best_loss_num_epochs_70_arcface.bin'
]
FOLDS = [0,1,3]
for wt,fold in zip(arcface,FOLDS):
    MODELS.append(get_arcface_model(wt,fold))
    WEIGHTS.append(1)
MODELS += [
    get_arcface_model('../input/shopee-models-arcface-full-data/sentence_transfomer_xlm_best_loss_num_epochs_24.bin',-1)
]
WEIGHTS.append(2)
for model,wt in zip(MODELS,WEIGHTS):
    print(model,wt)

In [ ]:
print(datetime.now()-start_time)

In [ ]:
!nvidia-smi

In [ ]:
model = sbert_ensemble(MODELS,WEIGHTS)
### Inference on Test
bert_neighbours = model.get_predictions(test_df,KNN=min(50,test_df.shape[0]),threshold=1)
print(bert_neighbours.shape)
bert_neighbours.columns = ['posting_id1', 'posting_id2', 'bert_distance']
bert_neighbours.head()

In [ ]:
bert_neighbours.to_csv('/tmp/bert_neighbours.csv',index=False)

In [ ]:
!nvidia-smi

In [ ]:
import gc
del model, MODELS, bert_neighbours, TOKENIZER,Adam,AdamW,ArcMarginProduct,DataLoader,ShopeeDataset,ShopeeNet,all_label_groups,data
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
print(datetime.now()-start_time)

In [ ]:
# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer 
# import os, re, csv, math, codecs
# import json

# stop_words = set(stopwords.words('english'))
# stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

# with open("../input/english-indonesian-data/indonesian_stopwords.txt") as f:
#     id_stop = [line.rstrip('\n') for line in f]
    
# stop_words.update(id_stop)

# with open('../input/english-indonesian-data/id_en_vocab.json', 'r') as fp:
#     id_en_vocab = json.load(fp)

In [ ]:
# USE_CLEAN_TEXT = True

# def preprocess_text(text):
#     s = str(text).lower()
#     # replace & with and
#     s = re.sub('&', ' and ', s)
#     # replace / with or (idn)
#     s = re.sub('/', 'atau', s, count=1)
#     # remove all special characters
#     s = s = re.sub(r"[^a-zA-Z0-9]+", ' ', s)
#     # replace 's with only s (the special character ' is not the standard one, hence the implementation)
#     s = re.sub(' s ', 's ', s)
#     # add whitespace after each number
#     s = re.sub(r"([0-9]+(\.[0-9]+)?)", r" \1 ", s).strip()
    
#     return s

# def id_en(t):
#     if t in id_en_vocab.keys():
#         return id_en_vocab[t]
#     else:
#         return t

# # def preprocess_text

# def remove_stopwords(text):
#     s = str(text).lower()
#     s = " ".join([id_en(word) for word in s.split() if word not in stop_words])
#     return s


# # def remove_stopwords

# def preprocess_row(row, col, func):
#     return func(row[col])


# # def preprocess_row

# def preprocess_text_df(df, txt_cols=['title_1', 'title_2'], func=preprocess_text):
#     txt_df = df[txt_cols].copy()
#     for col in txt_cols:
#         txt_df[col] = txt_df.apply(lambda x: preprocess_row(x, col, func=func), axis=1)
#     return txt_df


# test_df['title'] = preprocess_text_df(test_df, txt_cols=['title'])
# if USE_CLEAN_TEXT:
#     test_df['title'] = preprocess_text_df(test_df, txt_cols=['title'], func=remove_stopwords)

In [ ]:
tf_idf_neighbours = get_tf_idf_predictions(test_df)
tf_idf_neighbours.columns = ['posting_id1', 'posting_id2', 'tfidf_distance']
print(tf_idf_neighbours.shape)
tf_idf_neighbours.head()

In [ ]:
tf_idf_neighbours.to_csv('/tmp/tf_idf_neighbours.csv',index=False)

In [ ]:
del tf_idf_neighbours,test_df
gc.collect()

In [ ]:
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

In [ ]:
!ls -lh /tmp/

In [ ]:
print(datetime.now()-start_time)

## Ensemble

In [ ]:
import dask.dataframe as dd

In [ ]:
bert_neighbours = dd.read_csv('/tmp/bert_neighbours.csv')
tf_idf_neighbours = dd.read_csv('/tmp/tf_idf_neighbours.csv')
text_neighbours = dd.merge(bert_neighbours,tf_idf_neighbours,on=['posting_id1','posting_id2'],how='outer').fillna(1)
text_neighbours['text_distance'] = (text_neighbours['bert_distance'] + text_neighbours['tfidf_distance'])/2
del bert_neighbours,tf_idf_neighbours
gc.collect()

image_neighbours = dd.read_csv('/tmp/eff_neighbours.csv')
image_neighbours = image_neighbours.rename(columns={'eff_distance':'image_distance'})
# nfnet_neighbours = dd.read_csv('/tmp/nfnet_neighbours.csv')
# image_neighbours = dd.merge(eff_neighbours,nfnet_neighbours,on=['posting_id1','posting_id2'],how='outer').fillna(1)
# image_neighbours['image_distance'] = 0.65*image_neighbours['eff_distance'] + 0.35*image_neighbours['nfnet_distance']
# del eff_neighbours,eff_neighbours
# gc.collect()

# clip_neighbours = pd.read_csv('/tmp/clip_neighbours.csv')

In [ ]:
# top2_text = text_neighbours.sort_values('text_distance').groupby('posting_id1').head(2).reset_index(drop=True)
# intersection = pd.merge(image_neighbours,text_neighbours,on=['posting_id1','posting_id2'])
# intersection = intersection[(intersection.text_distance<0.3)&(intersection.image_distance<0.4)]

In [ ]:
neighbours = dd.merge(image_neighbours,text_neighbours,on=['posting_id1','posting_id2'],how='outer').fillna(1)
neighbours['distance'] = (neighbours.text_distance+neighbours.image_distance)/2

In [ ]:
del image_neighbours,text_neighbours
gc.collect()

In [ ]:
neighbours = neighbours[['posting_id1','posting_id2','image_distance','text_distance','distance']]

In [ ]:
%%time
neighbours = neighbours.compute()

In [ ]:
top2_imtext = neighbours.sort_values('distance').groupby('posting_id1').head(2).reset_index(drop=True)

In [ ]:
neighbours = pd.concat([
    neighbours[neighbours.distance<0.35],
    neighbours[neighbours.text_distance<0.18],
    neighbours[neighbours.image_distance<0.28],
]).drop_duplicates(['posting_id1','posting_id2'],keep='first')
neighbours_value_counts = neighbours.posting_id1.value_counts()
left_ids = neighbours_value_counts.loc[neighbours_value_counts==1].index.to_list()
neighbours = pd.concat([neighbours,top2_imtext[top2_imtext.posting_id1.isin(left_ids)]]).drop_duplicates(['posting_id1','posting_id2'],keep='first')
print(neighbours.shape)

In [ ]:
# neighbours = pd.concat([
#     text_neighbours[text_neighbours.text_distance<0.2],
#     top2_text[top2_text.text_distance<0.7],
#     image_neighbours[image_neighbours.image_distance<0.3],
#     intersection
# ]).drop_duplicates(['posting_id1','posting_id2'],keep='first')
# print(neighbours.shape)

In [ ]:
submission_df = neighbours.groupby('posting_id1').posting_id2.apply(lambda x:" ".join(x)).reset_index()
submission_df.columns = ['posting_id','matches']
submission_df.to_csv('submission.csv',index=False)
submission_df.head()